In [1]:
import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from bson.son import SON
from bson.code import Code
import json
import pandas as pd
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
# These are packages need for natural language processing:
import nltk
from __future__ import division, unicode_literals 
from textblob import TextBlob as tb
import math
from datetime import datetime
from dateutil import parser

In [2]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import amazon_data_sort 
from amazon_data_sort import get_b_product, get_reviews
import doc_summarization as ds

In [3]:
# #For Topic Modelling LDA:
# import lda
# #import scipy
# import gensim

# # from gensim import corpora, models, similarities
# # from gensim.models import hdpmodel, ldamodel
# # from itertools import izip

In [3]:
import reviews as r
import product_user_profile as pup
import doc_summarization as ds
import amazon_data_sort as ads

In [4]:
client = MongoClient()
mbf = client.dsbc.mbf
product_info = client.dsbc.product_info
name = product_info.find({}, {"_id":0,"asin":1,"title":1,"price":1})
beauty_list  = {}
for id_num in name:
    try:
        beauty_list[id_num["asin"]]=[id_num["title"],id_num["price"]]
    except:
        pass
#Creating database for reviews beauty file
rbf = client.dsbc.rbf

In [6]:
#These codes help find all sentence with the word of interests
#It's help full to see the exact condition surrounding the word
#to built all the terms for a category
from nltk import tokenize
for index in xrange(10):
    a = r.reviews(beauty_list,index)['Reviews']
    for i in a:
        sentences  = tokenize.sent_tokenize(i[0])
        for sentence in sentences:
            if "sun" in sentence:
                print sentence
                print ""
                print ""

As much as any of us hate to admit, how well we age is determined predominately by genetics plus exercise, good nutrition, hydration, sunscreen and regular skin care.


I will tell you, using this and staying out of the sun have protected my facial skin so much that, at age 56, I still have no wrinkles unless I smile really hard and you look up close.


"If you use this cream every day and stay out of the sun faithfully and/or use sunscreen, you will have beautiful skin long after your friends have started to get very noticeable lines.Regards,Karen


One of the few I've tried that doesn't smell like it contains sunscreen.


It's tint-free, which I also like.I use it primarily on my face and lower arms where the sun hits.




In [24]:
print summarize_item(beauty_list,9)
i=9
print r.reviews(beauty_list,i)['Reviews'][0]
pup.product_profiler(r.reviews(beauty_list,i))

{'dry_skin_score': 0.0, 'oily_skin_score': 0.0, u'name': u'DCL Daily Moisture SPF 15 3.5 oz', 'sunscreen_score': 0.0196078431372549, 'sensitive_score': 0.0, u'price': 32.53, 'combination_skin_score': 0.0, u'item_id': u'B000IOQ84U', 'redness_score': 0.0, 'anti_aging_score': 0.0, 'acne_score': 0.0}
[u"Love this moisturizer.  One of the few I've tried that doesn't smell like it contains sunscreen.  It's tint-free, which I also like.I use it primarily on my face and lower arms where the sun hits.  A single tube lasts quite awhile.", 5.0]


{'acne_score': 0.0,
 'anti_aging_score': 0.0,
 'combination_skin_score': 0.0,
 'dry_skin_score': 0.0,
 'oily_skin_score': 0.0,
 'redness_score': 0.0,
 'sensitive_score': 0.0,
 'sunscreen_score': 0.0196078431372549}

###Now transfer everything onto MongoDB

In [17]:
item_profile = client.dsbc.item_profile

In [21]:
def summarize_item(item_list,item_index):
    all_item_info = {}
    all_item_info = {"item_id": item_list.keys()[item_index],"name":item_list.values()[item_index][0],
                     "price":item_list.values()[item_index][1]}
    all_item_score = pup.product_profiler(r.reviews(item_list,item_index))
    all_item_info.update(all_item_score)
    return all_item_info

In [33]:
summarize_item(beauty_list, 5)

{'acne_score': 0.0,
 'anti_aging_score': 0.0,
 'combination_skin_score': 0.0,
 'dry_skin_score': 0.0,
 u'item_id': u'B00BJEJY9E',
 u'name': u'SK II Men UV Protect Moisturizer SPF 30 PA +++ New 50g / 1.7oz',
 'oily_skin_score': 0.0,
 u'price': 98.11,
 'redness_score': 0.0,
 'sensitive_score': 0.0,
 'sunscreen_score': 0.0}

In [21]:
# These code is to save all item information 
# into mongodb called item_profile

# for i in xrange(5570):
#     beauty_item = summarize_item(beauty_list, i)
#     item_profile.insert_one(beauty_item)

In [15]:
# sp = client.dsbc.smarter_profile
# for i in xrange(5570):
#      beauty_item = summarize_item(beauty_list, i)
#      sp.insert_one(beauty_item)

In [103]:
x = smarter_profile.find({"dry_skin_score": {"$gt": 0.04}, "oily_skin_score":{"$gt":0.01}})
list_b = []
for i in xrange(1):
    list_b.append(x.sort("dry_skin_score", pymongo.DESCENDING)[i])

list_b

[{u'_id': ObjectId('558332463cae551ac7dfcab5'),
  u'acne_score': 0.25812619502868067,
  u'anti_aging_score': 0.04780114722753346,
  u'combination_skin_score': 0.02676864244741874,
  u'dry_skin_score': 0.3652007648183556,
  u'item_id': u'B000YJ2SLG',
  u'name': u'CeraVe Moisturizing Lotion, 12-Ounce Bottle',
  u'oily_skin_score': 0.040152963671128104,
  u'price': 12.78,
  u'redness_score': 34.0,
  u'sensitive_score': 0.26577437858508607,
  u'sunscreen_score': 0.022944550669216062}]

In [30]:
def list_of_15(priority,concern_list):
    dd ={"dry_skin_score":-0.1,"oily_skin_score":-0.1, 
         "sensitive_score":-0.1,"sunscreen_score":-0.1,
         "combination_skin_score":-0.1,"redness_score":-0.1,
         "anti_aging_score":-0.1,"acne_score":-0.1}

# concern_list = ["dry_skin_score", "acne_score"]
    for concern in concern_list:
        dd[concern]=0.0
    
    sp = MongoClient().dsbc.smarter_profile
    x = sp.find({"dry_skin_score":{"$gt":dd["dry_skin_score"]}, "oily_skin_score":{"$gt":dd["oily_skin_score"]},
                 "sensitive_score":{"$gt":dd["sensitive_score"]},"sunscreen_score":{"$gt":dd["sunscreen_score"]},
                 "combination_skin_score":{"$gt":dd["combination_skin_score"]}, "redness_score":{"$gt":dd["redness_score"]},
                 "anti_aging_score":{"$gt":dd["anti_aging_score"]},"acne_score":{"$gt":dd["acne_score"]}})
    number_of_items = min(15,x.count())
    list_of_items = []
    for i in xrange(number_of_items):
        list_of_items.append(x.sort(priority,pymongo.DESCENDING)[i])
    
    return list_of_items       

In [29]:
x = sp.find({"dry_skin_score":{"$gt":0.0}, "oily_skin_score":{"$gt":-0.1},
                 "sensitive_score":{"$gt":0.0},"sunscreen_score":{"$gt":-0.1},
                 "combination_skin_score":{"$gt":-0.1}, "redness_score":{"$gt":-0.1},
                 "anti_aging_score":{"$gt":-0.1},"acne_score":{"$gt":-0.1}})
print x[1]

{u'dry_skin_score': 0.07368421052631578, u'oily_skin_score': 0.03684210526315789, u'name': u'Alba BotanicaTM Even Advanced Natural Moisturizer Sea Moss SPF 15 -- 2 fl oz', u'sunscreen_score': 0.08421052631578947, u'sensitive_score': 0.08421052631578947, u'price': 5.89, u'combination_skin_score': 0.0, u'item_id': u'B00005REAQ', u'_id': ObjectId('55887123e0a3ba27352051ab'), u'redness_score': 0.005263157894736842, u'anti_aging_score': 0.0, u'acne_score': 0.08421052631578947}


In [102]:
list_of_3("dry_skin_score",["dry_skin_score","oily_skin_score"])

{u'dry_skin_score': 0.0, u'oily_skin_score': 0.0, u'combination_skin_score': -0.1, u'sunscreen_score': -0.1, u'sensitive_score': -0.1, u'redness_score': -0.1, u'anti_aging_score': -0.1, u'acne_score': -0.1}


[]

In [30]:
x = sp.find({"dry_skin_score": {"$gt": 0.04}, "oily_skin_score":{"$gt":0.01}})
print x.sort("dry_skin_score", pymongo.DESCENDING)[1]
   

{u'dry_skin_score': 0.3620689655172414, u'oily_skin_score': 0.08045977011494253, u'name': u"POND'S Dry Skin Cream, 10.1-oz. (Pack of 3)", u'sunscreen_score': 0.0, u'sensitive_score': 0.06321839080459771, u'price': 17.03, u'combination_skin_score': 0.034482758620689655, u'item_id': u'B001ECQ4R6', u'_id': ObjectId('5588693ce0a3ba273520483a'), u'redness_score': 0.04597701149425287, u'anti_aging_score': 0.11494252873563218, u'acne_score': 0.1724137931034483}


###Putting reviews for product onto Mongo database

In [36]:
#This function return a dictionary including item name, summary of all reviews
#for that item and items rating average
def reviews_rating_summary(item_list, item_index):
    item_info = {"item_id": item_list.keys()[item_index],"name":item_list.values()[item_index][0]}
    item = ads.get_b_product(item_list, item_index)
    reviews_sum = ds.summarize(item['Reviews'])
    
    item_info['reviews_summary'] = reviews_sum['top_n_summary']
    
    star_rating  = ads.strict_rating(item['rating']['One_star'],
                                     item['rating']['Two_star'],
                                     item['rating']['Three_star'],
                                     item['rating']['Four_star'],
                                     item['rating']['Five_star'],
                                     3, 100)
    len_reviews = sum(item['rating'].values())
    
    item_info['number_of_reviews'] = len_reviews
    item_info['star_rating_ave'] = star_rating
    return item_info

In [56]:
reviews_rating_summary(beauty_list, 0)

{u'item_id': u'B004I9J816',
 u'name': u'H2O Aquafirm Micro-Collagen Moisturizer Unisex, 1.7 Ounce',
 u'number_of_reviews': 2,
 u'reviews_summary': [u'this one seems like it might be a winner.',
  u'it absorbed nicely and my skin did look and feel great after using it.',
  u"it did lose one star, though, because of the price- it's a bit steep.i bought it for a collagen moisturizer and immediately found it cleared my breakouts.",
  u'i still use an under eye cream since i am 56 but this cream is miraculous.',
  u"it soaks in so fast you can't believe it and it not greasy or shiny for someone with oily skin."],
 u'star_rating_ave': 3.0294117647058822}

In [32]:
item_id_list  = [i['item_id'] for i in list_of_15("dry_skin_score",["dry_skin_score", "oily_skin_score","acne_score"])]

item_id_list

[u'B000YJ2SLG',
 u'B001ECQ4R6',
 u'B004WC2H8A',
 u'B000BZB1FC',
 u'B0028N0NR2',
 u'B000PKWAWQ',
 u'B000WNLFBI',
 u'B00KCTER3U',
 u'B004OFYMW4',
 u'B000FQUGXA',
 u'B000VFUS0Q',
 u'B00ALB397Y',
 u'B00CM2AQE4',
 u'B00IT69F62',
 u'B00365DABC']

In [33]:
#Create a mongo database for the item's review info
item_reviews = client.dsbc.item_reviews

In [38]:
reviews_rating_summary(beauty_list, 1)

{u'item_id': u'B00C82YMWK',
 u'name': u'Beauty Renew 60+ Exclusive Day &amp; Night Cream for Mature Skin',
 u'number_of_reviews': 1,
 u'reviews_summary': [u'i am  65 and really like this cream.',
  u'it has been hard to find something that i feel really improves my skin without being greasy.',
  u'this is great under makeup and at bed time .',
  u'i was pleasantly surprised.',
  u'great price for product compared to other products that i paid much much for without good results.'],
 u'star_rating_ave': 3.01980198019802}

In [40]:
### Put all item_reviews on mongo database:

# for item_index in xrange(5570):
#      item_review_info = reviews_rating_summary(beauty_list, item_index)
#      item_reviews.insert_one(item_review_info)

###Put user sentiment for each items onto Mongo database:

In [46]:
import item_sentiment as i_sent

In [51]:
item_sent = MongoClient().dsbc.item_sent
# for key in beauty_list.keys():
#     sentiment_dict = {"item_id":key}
#     sentiment_dict["sentiment"] = i_sent.get_item_sentiment(key)
#     item_sent.insert_one(sentiment_dict)# 

In [49]:
a = beauty_list.keys()[:2]
for k_a in a:
    sent = {"item_id":k_a}
    sent["sentiment"]=i_sent.get_item_sentiment(k_a)
    print sent

{u'item_id': u'B004I9J816', u'sentiment': [[datetime.datetime(2014, 4, 9, 0, 0), datetime.datetime(2013, 1, 1, 0, 0)], [0.3722222222222222, 0.2]]}
{u'item_id': u'B00C82YMWK', u'sentiment': [[datetime.datetime(2014, 7, 9, 0, 0)], [0.28703703703703703]]}


5570

In [9]:
def skin_type_counter(list_of_text):
    dry_skin = 0
    oily_skin = 0
    combination_skin = 0
    sensitivity = 0
    total_reviews = len(list_of_text)
    for review_text in list_of_text:
        new_text = review_text[0].lower()
        dry_skin += new_text.count("dry skin")
        oily_skin += new_text.count("oily skin")
        combination_skin += new_text.count("combination skin")
        sensitivity += new_text.count("sensitive")*tb(new_text).sentiment.polarity
    return [dry_skin, oily_skin, combination_skin, sensitivity/(total_reviews+1)]
    

In [11]:
a = get_reviews(beauty_list,42)['Reviews']
# for i in a:
#     print i
#     print skin_type_counter([i])
#     print ""
#     print ""

In [15]:
words = []
sentences  = nltk.word_tokenize(a[0][0])

In [28]:
stopwords = nltk.corpus.stopwords.words("english")\
            + ['usps','shipping','$','#', 'very', 'amazon','every',
                '&','order','seller','packaging','manufacture', 'ever',
                'package','cute','delivery', '.',',',"return", "might"
                '?','...','day','week', 'weeks']


In [29]:
for sentence in sentences:
    tokens = nltk.word_tokenize(sentence)
    text = [word for word in tokens if word not in stopwords]
    tagged_text = nltk.pos_tag(text)
    for word, tag in tagged_text:
        words.append({"word": word, "pos": tag})

In [32]:
from nltk.stem.wordnet import WordNetLemmatizer

In [41]:
nouns = []
lem = WordNetLemmatizer()
noun_only = [noun for noun in words if noun["pos"] in ["NN", "NNS"]]
for noun_word in noun_only:
    nouns.append(lem.lemmatize(noun_word["word"]))
    

In [44]:
import logging
import gensim
from gensim import corpora
from gensim.corpora import BleiCorpus

In [18]:
a = [1]+[2]+[2,3]

In [20]:
list(set(a))

[1, 2, 3]